## Preprocessing

In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN','NAME','STATUS','SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [9]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [10]:
income_count= application_df['INCOME_AMT'].value_counts()
income_count

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [11]:
# Create list of income amounts to replace (combining all bins over 1 Million)
income_amt_to_replace = ['1M-5M', '10M-50M','5M-10M','50M+']

# Replace in dataframe
for income in income_amt_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(income,"1M+")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M+               1519
1-9999             728
10000-24999        543
Name: count, dtype: int64

In [12]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_counts[app_counts < 1000].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [14]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [15]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts <1800].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [17]:
application_df.nunique()

APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION         6
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             6
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df, dtype = int)
application_dummies_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999
0,5000,1,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,5000,0,0,0,0,0,1,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [19]:
# Reviewinig list of columns to identify columns to remove (for categorical need 1 less than number of categories)
application_dummies_df.columns

Index(['ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_1M+', 'INCOME_AMT_25000-99999'],
      dtype='object')

In [20]:
# Drop addditional columns to remove categorical redundancies
application_dummies_df = application_dummies_df.drop(columns = ['APPLICATION_TYPE_Other','AFFILIATION_Other','CLASSIFICATION_Other','USE_CASE_Other','ORGANIZATION_Trust',
                                                                'INCOME_AMT_1-9999'])

In [21]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop (columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Reviewing feature data columns and checking number of model inputs (#columns)
X.head()

,ASK_AMT,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,INCOME_AMT_0,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999
0,5000,0,0,0,0,0,0,0,1,0,...,0,1,1,0,0,1,0,0,0,0
1,108590,0,1,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
2,5000,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,1,0,0,0,0
3,6692,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,142590,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer - #pre-processed dataset has 28 inputs/columns
nn.add(tf.keras.layers.Dense(units=40, activation="leaky_relu", input_dim=len(X_train_scaled[0])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="leaky_relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 40)                1160      
                                                                 
 dense_22 (Dense)            (None, 25)                1025      
                                                                 
 dense_23 (Dense)            (None, 15)                390       
                                                                 
 dense_24 (Dense)            (None, 1)                 16        
                                                                 
Total params: 2591 (10.12 KB)
Trainable params: 2591 (10.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=70)


Epoch 1/70
804/804 [==============================] - 4s 3ms/step - loss: 0.5805 - accuracy: 0.7148
Epoch 2/70
804/804 [==============================] - 2s 3ms/step - loss: 0.5615 - accuracy: 0.7265
Epoch 3/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7285
Epoch 4/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7310
Epoch 5/70
804/804 [==============================] - 3s 3ms/step - loss: 0.5545 - accuracy: 0.7318
Epoch 6/70
804/804 [==============================] - 3s 4ms/step - loss: 0.5536 - accuracy: 0.7315
Epoch 7/70
804/804 [==============================] - 3s 3ms/step - loss: 0.5536 - accuracy: 0.7315
Epoch 8/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7326
Epoch 9/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7331
Epoch 10/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7320

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5558 - accuracy: 0.7269 - 923ms/epoch - 3ms/step
Loss: 0.5558196306228638, Accuracy: 0.7268804907798767


In [46]:
# Loading library to connect google drive in order to save model directly in drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [47]:
#Saving model in Google Drive Folder as HDF5 file
nn.save("/content/gdrive/MyDrive/Colab_Notebooks/AlphabetSoupCharity_Optimization3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
